In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from datetime import datetime

# importing csv files.
crimes_19 = pd.read_csv('input_data/Chicago_Crimes_2019.csv')
crimes_20 = pd.read_csv('input_data/Chicago_Crimes_2020.csv')
crimes_21 = pd.read_csv('input_data/Chicago_Crimes_2021.csv')

# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

print(len(crimes_data))

In [ ]:
# converting Date column in dataframe to date format and adding columns for month, year, day, time
crimes_data["Date"] = pd.to_datetime(crimes_data['Date'])
crimes_data["Day"] = crimes_data['Date'].dt.day_name()
crimes_data["Month Day"] =crimes_data['Date'].dt.day
crimes_data["Month"] = crimes_data['Date'].dt.month_name()
crimes_data["Year"] = crimes_data['Date'].dt.year
crimes_data["Time"] =crimes_data['Date'].dt.time
crimes_data["AM_PM"] = pd.to_datetime(crimes_data['Date']).dt.strftime("%p")
crimes_data.dropna(how='any', inplace=True)
crimes_data

In [ ]:
crime_type_overall = crimes_data.groupby(["Primary Type"])
overall_crime_type_df = pd.DataFrame(crimes_data["Primary Type"].value_counts())
overall_crime_type_df

In [ ]:
overall_crime_type_df = overall_crime_type_df.rename(
    columns={"Primary Type": "Crime Count"})
overall_crime_type_df

In [ ]:
top_ten_crimes = ["Theft","Battery","Criminal Damage","Assault","Deceptive Practice","Other Offense","Motor Vehicale Theft","Narcotics","Burglary","Robbery"]
crime_count = [141158,131204,76153,59061,48198,42760,29354,26354,24951,23740]
x_axis = np.arange(len(crime_count))

In [ ]:
plt.bar(x_axis, crime_count, color="b", align="center")
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, top_ten_crimes, rotation="vertical")
plt.title("Top Ten Most Common Crimes over the Past Three Years")
plt.xlabel("Crime Categories")
plt.ylabel("Number of Criminal Acts Committed")



plt.show()
plt.tight_layout()

In [ ]:
least_ten_crimes = ["Concealed Carry License Violation","Intimidation","Kidnapping","Gambling","Obscenity","Human Trafficking","Public Indecency","Other Narcotic Violation","Non-Criminal","Ritualism"]
least_crime_count = [534,441,377,180,156,30,24,14,8,1]
x_axis = np.arange(len(least_crime_count))


In [ ]:
plt.bar(x_axis, least_crime_count, color="r", align="center")
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, least_ten_crimes, rotation="vertical")
plt.title("Least Common Crimes over the Past Three Years")
plt.xlabel("Crime Categories")
plt.ylabel("Number of Criminal Acts Committed")



plt.show()
plt.tight_layout()

In [ ]:
number_crimes = crimes_data.groupby("Year")
yearly_crimes_df = pd.DataFrame(number_crimes["ID"].nunique())
yearly_crimes_df

In [ ]:
yearly_crimes_df = yearly_crimes_df.rename(
    columns={"ID": "Crime Count"})
yearly_crimes_df

In [ ]:
x = [2019,2020,2021]
y = [258143,207427,202350]
labels = ['2019', '2020', '2021']
 
# Plotting x-axis and y-axis
plt.plot(x, y)
 
# naming of x-axis and y-axis
plt.xlabel("Year")
plt.ylabel("Number of Criminal Acts Committed")
 
# naming the title of the plot
plt.title("Yearly Reported Crime Totals")
 
# setting x-axis values
plt.xticks(x, labels)
 
plt.show()


In [ ]:
crime_type = crimes_data.groupby("Year")

crime_type_df = pd.DataFrame(crime_type["Primary Type"].value_counts())

crime_type_df



In [ ]:
crime_type_df = crime_type_df.rename(
    columns={"Primary Type": "Crime Count"})
crime_type_df


In [ ]:
crime_type_df["Crime Count"].sum()

In [ ]:
most_common_crimes = crime_type_df.sort_values(["Crime Count"],
                                           ascending=False)
most_common_crimes.head(30)